In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time 

from citipy import citipy

import gmaps.datasets

# Import the API keys.
from config import weather_api_key
from config import g_key


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coord = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coord:
    print(coordinate[0], coordinate[1])

-14.856934504154168 128.56247729833314
-41.00387366161003 -41.271100602132975
57.7527090179089 -81.6907190551967
39.10543807625817 81.78605296324929
8.231837421575108 139.03763284852363
42.2177616584365 31.40981595667975
-86.96682751710466 -30.616438918859103
74.12278815873535 -22.39644154485427
21.01349126756034 38.41330715638426
14.87157540608942 162.39322708278763
-38.22226812498626 -131.62890737908327
-72.68138693633549 53.871301139024524
53.101110314942105 51.41743919594282
-2.1136453547347003 -179.60888225277955
34.990127010066175 26.130140899834828
-73.60769573854655 140.20314855094347
-14.194562676811643 -76.15883476067575
84.27739697440407 -175.96728601605793
-2.158119923824387 75.73154311079128
-2.3524660038342375 -150.80837121359158
-53.428940773658454 82.55469851127242
-64.86561871857515 -64.49229469220799
-12.979693228025837 -123.0279067465896
49.85361095478163 94.0281792269497
-61.983902116082184 -68.5436941403799
86.60867757133249 118.0979030819069
-53.01050611639457 163

20.42175851809489 -81.0835108052517
2.196804750236666 -17.196355132799198
-73.76340513813304 -24.13437124615777
-71.713945349784 126.25391941326609
66.98324718954288 37.47684333523995
42.25648846793732 128.40083398710078
36.142821827515704 113.19716042903644
-0.3508035754666565 -142.56412201672953
31.85912284564 27.867533130041153
26.783929925460995 -21.852907316588897
-6.870546504483414 82.73403709321065
-39.613977842620315 26.01819953181098
75.2675163439192 56.457625168886324
-21.883544727390387 -163.82018631913576
59.77056067234611 -172.4978850193381
5.536063035868736 -114.1112112229136
68.63134100948028 60.47177357506368
88.25192822152246 28.731911596503693
-63.18982341840841 -100.35558764217345
62.9735003819593 -123.0022177877359
-67.21517037668647 -95.48556661206922
59.39092702407996 99.92031396070558
81.60874127876897 -108.14293959015718
-85.55728898258131 -81.60530019349754
-46.686544034727724 136.97751107562868
-86.31444292380482 52.93032521281182
-5.331352170032687 -173.51264

In [5]:
# Get the nearest city using the citipy module and create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coord:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

740

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Make a 'Get' request for the city weather to check good response.
city_weather = requests.get(url)
city_weather

<Response [400]>

In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_min_temp = city_weather["main"]["temp_min"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_descr = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_descr,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kununurra
City not found. Skipping...
Processing Record 2 of Set 1 | cidreira
City not found. Skipping...
Processing Record 3 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 4 of Set 1 | aksu
City not found. Skipping...
Processing Record 5 of Set 1 | airai
City not found. Skipping...
Processing Record 6 of Set 1 | zonguldak
City not found. Skipping...
Processing Record 7 of Set 1 | ushuaia
City not found. Skipping...
Processing Record 8 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 9 of Set 1 | jiddah
City not found. Skipping...
Processing Record 10 of Set 1 | butaritari
City not found. Skipping...
Processing Record 11 of Set 1 | rikitea
City not found. Skipping...
Processing Record 12 of Set 1 | east london
City not found. Skipping...
Processing Record 13 of Set 1 | bogatoye
City not found. Skipping...
Processing Record 14 of Set 1 | vaitupu
City

City not found. Skipping...
Processing Record 19 of Set 3 | puerto ayora
City not found. Skipping...
Processing Record 20 of Set 3 | shingu
City not found. Skipping...
Processing Record 21 of Set 3 | stornoway
City not found. Skipping...
Processing Record 22 of Set 3 | santa ines
City not found. Skipping...
Processing Record 23 of Set 3 | puerto escondido
City not found. Skipping...
Processing Record 24 of Set 3 | chokurdakh
City not found. Skipping...
Processing Record 25 of Set 3 | faanui
City not found. Skipping...
Processing Record 26 of Set 3 | port macquarie
City not found. Skipping...
Processing Record 27 of Set 3 | gelemso
City not found. Skipping...
Processing Record 28 of Set 3 | qiongshan
City not found. Skipping...
Processing Record 29 of Set 3 | college
City not found. Skipping...
Processing Record 30 of Set 3 | kruisfontein
City not found. Skipping...
Processing Record 31 of Set 3 | avarua
City not found. Skipping...
Processing Record 32 of Set 3 | malanje
City not found.

City not found. Skipping...
Processing Record 38 of Set 5 | lar
City not found. Skipping...
Processing Record 39 of Set 5 | peterhead
City not found. Skipping...
Processing Record 40 of Set 5 | grindavik
City not found. Skipping...
Processing Record 41 of Set 5 | ahipara
City not found. Skipping...
Processing Record 42 of Set 5 | price
City not found. Skipping...
Processing Record 43 of Set 5 | hofn
City not found. Skipping...
Processing Record 44 of Set 5 | port-gentil
City not found. Skipping...
Processing Record 45 of Set 5 | flin flon
City not found. Skipping...
Processing Record 46 of Set 5 | kropotkin
City not found. Skipping...
Processing Record 47 of Set 5 | batagay
City not found. Skipping...
Processing Record 48 of Set 5 | udachnyy
City not found. Skipping...
Processing Record 49 of Set 5 | duluth
City not found. Skipping...
Processing Record 50 of Set 5 | lipin bor
City not found. Skipping...
Processing Record 1 of Set 6 | khatanga
City not found. Skipping...
Processing Reco

City not found. Skipping...
Processing Record 8 of Set 8 | lundazi
City not found. Skipping...
Processing Record 9 of Set 8 | amapa
City not found. Skipping...
Processing Record 10 of Set 8 | belyy yar
City not found. Skipping...
Processing Record 11 of Set 8 | armacao dos buzios
City not found. Skipping...
Processing Record 12 of Set 8 | asosa
City not found. Skipping...
Processing Record 13 of Set 8 | kothi
City not found. Skipping...
Processing Record 14 of Set 8 | karamay
City not found. Skipping...
Processing Record 15 of Set 8 | ca mau
City not found. Skipping...
Processing Record 16 of Set 8 | nikolskoye
City not found. Skipping...
Processing Record 17 of Set 8 | okato
City not found. Skipping...
Processing Record 18 of Set 8 | tezu
City not found. Skipping...
Processing Record 19 of Set 8 | kuopio
City not found. Skipping...
Processing Record 20 of Set 8 | namibe
City not found. Skipping...
Processing Record 21 of Set 8 | umarkot
City not found. Skipping...
Processing Record 22

City not found. Skipping...
Processing Record 27 of Set 10 | tlyarota
City not found. Skipping...
Processing Record 28 of Set 10 | tarija
City not found. Skipping...
Processing Record 29 of Set 10 | srednekolymsk
City not found. Skipping...
Processing Record 30 of Set 10 | abu kamal
City not found. Skipping...
Processing Record 31 of Set 10 | jarjis
City not found. Skipping...
Processing Record 32 of Set 10 | hilton head island
City not found. Skipping...
Processing Record 33 of Set 10 | sorkjosen
City not found. Skipping...
Processing Record 34 of Set 10 | castlegar
City not found. Skipping...
Processing Record 35 of Set 10 | kemijarvi
City not found. Skipping...
Processing Record 36 of Set 10 | luangwa
City not found. Skipping...
Processing Record 37 of Set 10 | kango
City not found. Skipping...
Processing Record 38 of Set 10 | nalut
City not found. Skipping...
Processing Record 39 of Set 10 | ajdabiya
City not found. Skipping...
Processing Record 40 of Set 10 | poum
City not found. 

City not found. Skipping...
Processing Record 45 of Set 12 | trelew
City not found. Skipping...
Processing Record 46 of Set 12 | vila velha
City not found. Skipping...
Processing Record 47 of Set 12 | hobyo
City not found. Skipping...
Processing Record 48 of Set 12 | hibbing
City not found. Skipping...
Processing Record 49 of Set 12 | navolato
City not found. Skipping...
Processing Record 50 of Set 12 | polyanka
City not found. Skipping...
Processing Record 1 of Set 13 | asyut
City not found. Skipping...
Processing Record 2 of Set 13 | nanakuli
City not found. Skipping...
Processing Record 3 of Set 13 | cabedelo
City not found. Skipping...
Processing Record 4 of Set 13 | zaysan
City not found. Skipping...
Processing Record 5 of Set 13 | ngukurr
City not found. Skipping...
Processing Record 6 of Set 13 | sao filipe
City not found. Skipping...
Processing Record 7 of Set 13 | abasolo
City not found. Skipping...
Processing Record 8 of Set 13 | itarema
City not found. Skipping...
Processing

City not found. Skipping...
Processing Record 13 of Set 15 | achit
City not found. Skipping...
Processing Record 14 of Set 15 | umm durman
City not found. Skipping...
Processing Record 15 of Set 15 | pleasanton
City not found. Skipping...
Processing Record 16 of Set 15 | xinzhi
City not found. Skipping...
Processing Record 17 of Set 15 | vengerovo
City not found. Skipping...
Processing Record 18 of Set 15 | norsup
City not found. Skipping...
Processing Record 19 of Set 15 | parana
City not found. Skipping...
Processing Record 20 of Set 15 | camara de lobos
City not found. Skipping...
Processing Record 21 of Set 15 | danville
City not found. Skipping...
Processing Record 22 of Set 15 | whittlesea
City not found. Skipping...
Processing Record 23 of Set 15 | waipawa
City not found. Skipping...
Processing Record 24 of Set 15 | dolbeau
City not found. Skipping...
Processing Record 25 of Set 15 | ocean city
City not found. Skipping...
Processing Record 26 of Set 15 | husavik
City not found. 

In [9]:
print ("The length of the Dictionary is", len(city_data))

The length of the Dictionary is 0


In [10]:
# Convert the array of dictionaries to a Pandas dataframe.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

""
